In [1]:
### If center=False (default setting), closed defaults to 'right'. This means the window is left-closed and right-open, i.e., the window includes points on the left boundary but excludes points on the right boundary.
### If center=True, closed defaults to 'both'. This means the window is closed on both sides, i.e., the window includes points on both the left and right boundaries.

import os
import sys
root_directory = os.getcwd()[0:3]
sys.path.append(root_directory+'SCI\\SCI9\\2_DataAnalysis\\04_code\\1_04_Soybean')
sys.path.append(r'C:\ProgramData\anaconda3\Lib\site-packages') 
sys.path.append(r'C:\Users\DELL\.conda\envs\myenv\Lib\site-packages') 
sys.path.append(r'C:\Users\DELL\.conda\envs\rasterio_env\Lib\site-packages') 
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
# Add new environment variables
import nbimporter 
from functions import spei, extreme_temperature, aggre_8days, aggre_8days_modis

T_upper = 34
T_lower = 0

root_directory = os.getcwd()[0:3]
crop = '02_wheat';
countryID = '06_India'
country = countryID[3:]
inputpath_base = root_directory + '\\SCI\\SCI9_1\\02_data\\'+crop+'\\'+countryID
inpath_dates_other = root_directory + '\\SCI\\SCI9_1\\02_data\\'+crop+'\\'+countryID+'\\'+'01_data'+'\\'+'07_Information'
other_infornamtion = pd.read_csv(os.path.join(inpath_dates_other, 'information.txt'), sep=' ', header=None)
startyear, endyear, shp_name = other_infornamtion.iloc[0,0], other_infornamtion.iloc[0,1], other_infornamtion.iloc[0,2]

startyear = 2001 # Model start year
yieldyear = endyear # Yield record start year
years = range(startyear, endyear+1)
years_str = [str(year) for year in years]

VIs =  [ '_KNDVI' ,'_EVI','_NDVI'] # Vegetation Indices
Cilmate = ['_Pre' ,'_Tmin' ,'_Solar','_Tmean','_Tmax'] # Meteorological variables
Climate_Exogenous  = ['_CDD' ,'_HDD' ,'_GDD','_VPD','_wind_speed','_SPEI'] # Exogenous climate variables (Cooling Degree Days, Heating Degree Days, Growing Degree Days, etc.)
soil_feature = [ 'SAND','AWC', 'SILT','ORG_CARBON',  'TOTAL_N', 'PH_WATER',  'CEC_SOIL', 'CLAY'] # Soil properties
loc_feature = ['elevation', 'lat', 'lon'] # Location features
Year_feature = ['year']; union_feature = ['idJoin']; # Common identifier
dynamic_features = [ '_KNDVI' ,'_EVI','_NDVI','_Pre' ,'_Tmin' ,'_Solar','_Tmean','_VPD', '_wind_speed' ,'_Tmax'] # Dynamic environmental features

C:\Users\DELL\.conda\envs\myenv\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
regions =  ['I']
for region in regions:
    pre_filename = 'WinterWheat_'+country+region+'_'
    dataall = pd.DataFrame()
    for year in years:
        data_new = pd.read_csv(os.path.join(inputpath_base, '01_data','04_GEEdownloadData','01_DailyData', pre_filename + str(year) + '.csv')).iloc[:, 1:]
        Tmin_columns = [col for col in data_new.columns if '_Tmin' in col]
        Tmin = data_new[Tmin_columns].values;
        Tmean_columns = [col for col in data_new.columns if '_Tmean' in col]
        Tmean = data_new[Tmean_columns].values;
        Tmax_columns = [col for col in data_new.columns if '_Tmax' in col]
        Tmax = data_new[Tmax_columns].values;
        Pre_columns = [col for col in data_new.columns if '_Pre' in col]
        Pre = data_new[Pre_columns].values;
        days = Pre.shape[1]
        dates = pd.date_range(start=str(year)+'-01-01', periods=days, freq='D')
        data_new['year'] = year
        # Calculate extreme meteorological indicators
        spei_df = spei(dates, Pre, Tmean)
        CDD_df, HDD_df, GDD_df = extreme_temperature(dates, Tmax, Tmin, T_upper, T_lower) # Note: Potential issue exists
        data_new1 = aggre_8days(dynamic_features, dates, data_new) # Aggregate daily data to 8-day intervals
        data_new1 = pd.concat([CDD_df, HDD_df, GDD_df, spei_df, data_new1, data_new[soil_feature+loc_feature+Year_feature+union_feature]], axis=1)
        dataall = pd.concat([dataall, data_new1])
    
    outputpath_base = os.path.join(inputpath_base, '01_data','05_buildmodel','01_weekdata')
    os.makedirs(outputpath_base, exist_ok=True)
    dataall.to_csv(os.path.join(outputpath_base, region+'_allweekdata.csv'), index=False)# Export combined 8-day interval data

In [3]:
#############################02_8-day MODIS data aggregation################################
dynamic_features_8sggre = ['_modis_Gpp','_modis_Fpar','_modis_LAI','_PML_Gpp'] # MODIS-derived dynamic features (Gross Primary Productivity, Fraction of Photosynthetically Active Radiation, Leaf Area Index)

for region in regions:
    pre_filename2 = 'WinterWheat'+ '_8agg' + country+region + '_'    
    dataall = pd.DataFrame()
    for year in years:
        data_new = pd.read_csv(os.path.join(inputpath_base, '01_data','04_GEEdownloadData','01_WeekotherData', pre_filename2 + str(year) + '.csv')).iloc[:, 1:]
        dates = pd.date_range(start=str(year)+'-01-01', end=str(year)+'-12-31', freq='8D')
        data_new1 = aggre_8days_modis(dynamic_features_8sggre, dates, data_new) # Aggregate MODIS data to 8-day intervals
        data_new1['year'] = year
        data_new1['idJoin'] = data_new['idJoin']
        dataall = pd.concat([dataall, data_new1])
    #dataall = dataall.dropna(how='all',axis=1)
    for col in dynamic_features_8sggre:
        dataname = [f'Week{week}{col}' for week in range(1, 47)] # Generate 46 8-day interval columns (365 days ≈ 46×8 days)
        dataall[dataname] = dataall[dataname].apply(lambda row: row.fillna(row.rolling(4, min_periods=1).mean()), axis=1) # Fill missing values with 4-period rolling mean
    dataall.fillna(0, inplace=True) # Fill remaining missing values with 0
    outputpath_base = os.path.join(inputpath_base, '01_data','05_buildmodel','01_weekdata', region+'_allweekdata_8modis.csv')
    dataall.to_csv(outputpath_base, index=False)

In [4]:
################03_Merge yield data with all environmental data by region########################
for region in regions:
    data_new1 = pd.read_csv(os.path.join(inputpath_base, '01_data','05_buildmodel','01_weekdata', region+'_allweekdata.csv'))
    data_new2 = pd.read_csv(os.path.join(inputpath_base, '01_data','05_buildmodel','01_weekdata', region+'_allweekdata_8modis.csv'))
    inputpath_base_Yield =  os.path.join(inputpath_base,'01_data', '01_Yield', country+'_winter.csv')
    long_df = pd.read_csv(inputpath_base_Yield)
    

    # Five-year moving average to capture long-term yield trends
    long_df = pd.read_csv(inputpath_base_Yield)
    long_df_mean = long_df.copy()
    data = long_df_mean.loc[:, str(yieldyear):].rolling(window=5, center=False, min_periods=2, closed='right', axis=1).mean() # 5-year moving average (excludes current year, right-closed window)
    long_df_mean.loc[:, str(yieldyear):] = data # Handle missing values for the starting year
    long_df_mean.loc[:, str(yieldyear)] = long_df_mean.loc[:, str(yieldyear+1)].values
    
    # Previous year's yield as potential yield
    long_df_pre = long_df.copy()
    data = long_df_pre.loc[:, str(yieldyear):str(endyear)].values[:, :-1] # Shift yield data to use previous year's value
    long_df_pre.loc[:, str(yieldyear+1):] = data
    long_df_pre.loc[:, str(yieldyear)] = long_df.loc[:, str(yieldyear+1)] # Fill missing value for the first year with next year's yield
    
    # Reshape yield data from wide to long format
    long_df = pd.melt(long_df, id_vars=['idJoin'], var_name='year', value_name='Yield', value_vars=['idJoin']+years_str)
    if long_df['year'].dtype == 'object':  
        long_df['year'] = long_df['year'].astype(int)  
        
   # long_df = long_df.drop(['Country'],axis=1)
    long_df_mean = pd.melt(long_df_mean, id_vars=['idJoin'], var_name='year', value_name='Mean_Yield', value_vars=['idJoin']+years_str)
    if long_df_mean['year'].dtype == 'object':  
        long_df_mean['year'] = long_df_mean['year'].astype(int)     
        
    long_df_pre = pd.melt(long_df_pre, id_vars=['idJoin'], var_name='year', value_name='Previous_Yield', value_vars=['idJoin']+years_str)
    if long_df_pre['year'].dtype == 'object':  
        long_df_pre['year'] = long_df_pre['year'].astype(int) 


    # Merge yield data with environmental data
    data_yield_ori_Visall = data_new1.merge(long_df, how='left', on=['idJoin', 'year']).dropna(axis=1, how='all')
    
    data_yield_ori_Visall = long_df_mean.merge(data_yield_ori_Visall, how='inner', on=['idJoin', 'year']).dropna(axis=1, how='all') # Merge with 5-year moving average yield
    data_yield_ori_Visall = data_new2.merge(data_yield_ori_Visall, how='inner', on=['idJoin', 'year']).dropna(axis=1, how='all') # Merge with MODIS 8-day data
    data_yield_ori_Visall = long_df_pre.merge(data_yield_ori_Visall, how='inner', on=['idJoin', 'year']).dropna(axis=1, how='all') # Merge with previous year's yield
    outputpath_base = os.path.join(inputpath_base, '01_data','05_buildmodel','01_weekdata', region+'_allweekYielddata.csv')
    data_yield_ori_Visall.to_csv(outputpath_base, index=False) # Export final merged dataset for model training

In [11]:
 # The previous algorithm uses the previous year's yield, so one additional year of data is required
# If historical yield data is available, calculate the moving average of previous years and use the previous year's yield
# If no historical data exists, use the average of subsequent years and the next year's yield as the reference



In [8]:

# Reassign values for 2003
# Merge datasets


,idJoin,year,Yield,PreYield
0,CHN_0,2003,2668.987300,989.000000
1,CHN_1,2003,2668.987300,1665.000000
2,CHN_1000,2003,2668.987300,1665.000000
3,CHN_1001,2003,2668.987300,1665.000000
4,CHN_1003,2003,2668.987300,1665.000000
...,...,...,...,...
24843,CHN_990,2018,2325.343986,2232.668566
24844,CHN_995,2018,2295.907660,2190.875233
24845,CHN_996,2018,1785.294118,1441.613588
24846,CHN_998,2018,1785.294118,1441.613588
